# **Códificación Shannon Fano**

https://en.wikipedia.org/wiki/Shannon%E2%80%93Fano_coding

*La codificación Shannon-Fano es un algoritmo de compresión de datos sin pérdida desarrollado por Robert Fano a partir de una idea de Claude Shannon.*

**Se trata de una codificación de entropía que produce un código de prefijo muy similar a un código de Huffman , aunque no siempre óptimo, a diferencia de este último.**

Un árbol Shannon-Fano se construye de acuerdo a una especificación diseñada para definir una tabla de códigos efectiva. El algoritmo actual es simple:

1. Para una lista de símbolos dada, crear su correspondiente lista de probabilidades o de frecuencias de aparición de manera que se conozca la frecuencia relativa de ocurrencia de cada símbolo.

2. Ordenar las listas de símbolos de acuerdo a la frecuencia, con los símbolos de ocurrencia más frecuente a la izquierda y los menos comunes a la derecha.

3. Dividir la lista en dos partes, haciendo la frecuencia total de la mitad izquierda lo más próxima posible a la de la mitad derecha.

4. Asignar a la mitad izquierda el dígito binario “0”, y a la mitad derecha el dígito “1”. Esto significa que los códigos para los símbolos en la primera mitad empezarán con “0”, y que los códigos de la segunda mitad empezarán por “1”.
5. Aplicar recursivamente los pasos 3 y 4 a cada una de las dos mitades, subdividiéndolas en grupos y añadiendo bits a los códigos hasta que cada símbolo se corresponde con una hoja del árbol.

### **1. Fuente de información**

In [246]:
# 1. Fuente de información (leyendo un archivo de texto)
def leer_archivo(nombre_archivo):
    try:
        with open(nombre_archivo, encoding="utf8") as archivo:
            contenido = archivo.read()
            return contenido
    except FileNotFoundError:
        print(f"El archivo '{nombre_archivo}' no fue encontrado.")
        return ""

nombre_archivo_fuente = "input.txt"
texto_original = leer_archivo(nombre_archivo_fuente)

### **2. Transmisor**

In [247]:
class Compresor: # Definimos una clase llamada "Compresor".
# Clase comprimida para almacenar diferentes parámetros de cadena.
    def __init__(self, caracter):
        self.original = caracter # Almacena el carácter original sin comprimir.
        self.contador = 0 # Se utiliza para llevar un registro del número de veces que aparece el carácter en la cadena comprimida.
        self.codigo = "" # Se utiliza para almacenar el código comprimido asociado al carácter.
        self.probabilidad = 0 # Se utiliza para almacenar la probabilidad de que el carácter aparezca en la cadena original.

In [248]:
# Esta función está destinada a ser utilizada para ordenar una lista de compresores según su probabilidad.
def obtener_probabilidad(compresores):
    return compresores.probabilidad

In [249]:
# Dividir probabilidades en orden utilizadas en la codificación shannon.
def divisor(probabilidad, puntero):
    # Calcula la diferencia entre la suma de las probabilidades en la parte izquierda del puntero y la suma de las probabilidades en la parte derecha del puntero. 
    diff = sum(probabilidad[:puntero+1]) - sum(probabilidad[puntero+1:])
    # Comprueba si la diferencia calculada es negativa, lo que implica que la división actual no es adecuada y debe intentar otra división.
    if diff < 0:
        # Llama recursivamente a la función divisor con el mismo conjunto de probabilidades, pero incrementa el puntero en 1.
        punto = divisor(probabilidad, puntero+1)
        diff_1 = sum(probabilidad[:punto]) - sum(probabilidad[punto:])
        diff_2 = sum(probabilidad[:punto+1]) - sum(probabilidad[punto+1:])
        # Compara las diferencias diff_1 y diff_2 y determina cuál de ellas es más pequeña en valor absoluto.
        if abs(diff_1) < abs(diff_2):
            return punto - 1
        return punto
    else:
        return puntero
    
texto_decomprimido = texto_original

In [250]:
# Se encarga de asignar códigos "0" y "1" a los compresores. 
def codificador(compresores, particion):
    if particion > 0: # Indica que todavía hay compresores que deben recibir códigos.
        parte_1 = compresores[:particion+1]
        for i in parte_1:
            i.codigo += '0' # Esto indica que estos compresores están en la parte izquierda de la partición.
        if len(parte_1) <= 1: # En este caso, retorna y no se realizan más divisiones.
            return
        codificador(parte_1, divisor(probabilidad=[i.probabilidad for i in parte_1], puntero=0))
        parte_2 = compresores[particion+1:]
        for i in parte_2:
            i.codigo += '1' # Esto indica que estos compresores están en la parte derecha de la partición.
        if len(parte_2) <= 1:
            return
        codificador(parte_2, divisor(probabilidad=[i.probabilidad for i in parte_2], puntero=0))
    elif particion == 0: # En caso de que la partición sea igual a cero, significa que todos los compresores se encuentran en la misma partición.
        parte_1 = compresores[:particion+1]
        for i in parte_1:
            i.codigo += '0'
        parte_2 = compresores[particion+1:]
        for i in parte_2:
            i.codigo += '1'

In [251]:
# Realiza el proceso de compresión de datos utilizando el algoritmo de Shannon-Fano.
def comprimir_datos(datos):
    procesados = [] # Crea una lista vacía llamada procesados para mantener un registro de los caracteres procesados.
    compresores = [] # Crea una lista vacía llamada compresores que almacenará objetos Compresor.
    longitud_total = len(datos)
    for i in range(len(datos)):
        if datos[i] not in procesados: # Verifica si el carácter actual no ha sido procesado previamente (no se encuentra en la lista procesados).
            procesados.append(datos[i]) # Agrega el carácter actual a la lista procesados para indicar que ha sido procesado.
            conteo = datos.count(datos[i]) # Cuenta cuántas veces aparece el carácter actual en la cadena de entrada y almacena el resultado en la variable conteo.
            var = conteo / longitud_total # Calcula la probabilidad del carácter actual dividiendo su conteo por la longitud total de la cadena.
            comp = Compresor(datos[i])
            comp.contador = conteo
            comp.probabilidad = var
            compresores.append(comp)
    compresores_ordenados = sorted(compresores, key=obtener_probabilidad, reverse=True) # Ordena la lista de compresores en orden descendente de probabilidad utilizando la función obtener_probabilidad como clave de ordenamiento. 
    particion = divisor(probabilidad=[i.probabilidad for i in compresores_ordenados], puntero=0)
    codificador(compresores_ordenados, particion)
    return compresores_ordenados


### **3. Canal**

In [252]:
# 3 Canal (se agregan ruido mediante la perdida de paquetes)

### **4. Receptor**

In [253]:
with open("output.txt", "w", encoding="utf-8") as archivo_salida:
    archivo_salida.write(texto_decomprimido)
print()
print("Mensaje decodificado guardado en 'output'.")


Mensaje decodificado guardado en 'output'.


### **5. Destino de información**

In [254]:
# 5. Destino de Información (Destinatario)
print("                |------------------------|")
print("                |    Datos codificados   |")
print("                |------------------------|")
print()
datos_comprimidos = comprimir_datos(texto_original)
for i in datos_comprimidos:
    print(f"Caracter: {i.original}, Código: {i.codigo}, Probabilidad: {i.probabilidad}")

                |------------------------|
                |    Datos codificados   |
                |------------------------|

Caracter:  , Código: 000, Probabilidad: 0.15129519877472197
Caracter: e, Código: 001, Probabilidad: 0.10741160018645535
Caracter: a, Código: 010, Probabilidad: 0.09149630418858627
Caracter: o, Código: 011, Probabilidad: 0.06412732236798295
Caracter: s, Código: 011, Probabilidad: 0.05946593860291669
Caracter: n, Código: 1000, Probabilidad: 0.05686888193380835
Caracter: r, Código: 1001, Probabilidad: 0.05147499500566025
Caracter: i, Código: 1001, Probabilidad: 0.040154491576213626
Caracter: u, Código: 1010, Probabilidad: 0.03715788772724246
Caracter: d, Código: 1011, Probabilidad: 0.036358793367516816
Caracter: l, Código: 1011, Probabilidad: 0.03535992541785976
Caracter: t, Código: 11000, Probabilidad: 0.03376173669840847
Caracter: c, Código: 11001, Probabilidad: 0.027169208230671905
Caracter: m, Código: 110100, Probabilidad: 0.02024372377971632
Caracter: p, C